In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_production_missing_category = pl.read_parquet(datadir / "production_missing_category.parquet")
pl_customer                    = pl.read_parquet(datadir / "customer.parquet")

# 11章 カテゴリ
## 11-1 カテゴリ型への変換
### Q: 性別をカテゴリに適したデータ型に変換


#### Awesome

In [ ]:
(
    pl_customer
    .select([
        pl.col("sex"),
        pl.col("sex").cast(pl.Categorical).alias("sex_cat")
    ])
)

## 11-2 カテゴリ値の欠損処理
### Q: $k$-NNによるカテゴリ値の補完


#### Awesome

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# （1） 欠損していないデータと欠損しているデータを分割
df_notnull = pl_production_missing_category.filter(pl.col("type").is_not_null())
df_null = pl_production_missing_category.filter(pl.col("type").is_null())

# （2） k-NNモデルのオブジェクトを作成
knn = KNeighborsClassifier(n_neighbors=3)
cols = ["length", "thickness"]

# （3） 欠損していないデータを用いてk-NNモデルを学習
knn.fit(df_notnull[cols], df_notnull["type"])

# （4） 欠損しているデータを用いて予測値を計算
pred = knn.predict(df_null[cols])

# （5） 欠損していないデータと欠損補完したデータを結合
pl.concat([df_notnull, df_null.with_columns(type=pl.Series(pred))])

## 11-3 カテゴリ値の集約
### Q: 年齢カテゴリの集約


#### Awesome

In [ ]:
(
    pl_customer
    .with_columns(
        age_cat=pl.when(pl.col("age") >= 60).then(6).otherwise(pl.col("age") / 10)
            .cast(int)
    )
)

## 11-4 カテゴリ値の組み合わせ
### Q: 性別と年齢のカテゴリの組み合わせ


#### Awesome

In [ ]:
(
    pl_customer
    # （1） ageのカテゴリを作成
    .with_columns(age_cat=(pl.col("age") / 10).cast(int))
    # （2） sexとageのカテゴリを結合
    .with_columns(sex_age_cat=
        (pl.col("sex").fill_null("null") + "_" + pl.col("age_cat").cast(pl.Utf8))
        .cast(pl.Categorical))
)

## 11-5 カテゴリ値の数値化
### Q: 性別のone-hotエンコーディング


#### Awesome

In [ ]:
from category_encoders.one_hot import OneHotEncoder

# （1） エンコーディングを行うオブジェクトを生成
encoder = OneHotEncoder(cols="sex", use_cat_names=True)
# （2） エンコーディングを実施
pl.DataFrame(encoder.fit_transform(pl_customer.to_pandas()))